In [21]:
import sqlite3
import random
from datetime import datetime, timedelta
import os
import pandas as pd

def create_and_populate_database():
    """
    Creates and populates an SQLite database named sales.db.
    The database contains three tables: customers, products, and sales.
    """
    # Check if the database file already exists and delete it
    if os.path.exists('sales1.db'):
        os.remove('sales1.db')
        print("Existing 'sales.db' file deleted.")

    # Connect to the database (creates the file if it doesn't exist)
    conn = sqlite3.connect('sales1.db')
    cursor = conn.cursor()

    # --- Create Tables ---
    # Create the customers table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS customers (
            customer_id INTEGER PRIMARY KEY,
            name TEXT NOT NULL,
            city TEXT,
            segment TEXT
        )
    ''')

    # Create the products table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS products (
            product_id INTEGER PRIMARY KEY,
            name TEXT NOT NULL,
            category TEXT,
            price REAL
        )
    ''')

    # Create the sales table, now with a revenue column
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS sales (
            sale_id INTEGER PRIMARY KEY,
            customer_id INTEGER,
            product_id INTEGER,
            quantity INTEGER,
            revenue REAL,
            sale_date TEXT,
            FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
            FOREIGN KEY (product_id) REFERENCES products(product_id)
        )
    ''')

    # --- Populate Tables with Sample Data ---
    # Sample data for customers
    customer_names = [
        "Alice Johnson", "Bob Smith", "Charlie Brown", "Diana Prince", "Edward Clark",
        "Fiona Gallagher", "George Lopez", "Hannah Miller", "Ian Wright", "Jessica Alba",
        "Kevin Hart", "Laura Prepon", "Michael Scott", "Nora Allen", "Oliver Queen",
        "Pam Beesly", "Quinn Fabray", "Rachel Green", "Steve Rogers", "Tina Fey"
    ]
    cities = ["New York", "Los Angeles", "Chicago", "Houston", "Phoenix"]
    segments = ["Consumer", "Corporate", "Home Office"]

    for name in customer_names:
        city = random.choice(cities)
        segment = random.choice(segments)
        cursor.execute("INSERT INTO customers (name, city, segment) VALUES (?, ?, ?)", (name, city, segment))

    # Sample data for products
    product_data = [
        ("Laptop", "Technology", 1200.00), ("Mouse", "Technology", 25.50),
        ("Office Chair", "Furniture", 250.00), ("Desk", "Furniture", 450.00),
        ("Pen Set", "Stationery", 15.00), ("Notebook", "Stationery", 5.99),
        ("Coffee Machine", "Appliances", 150.00), ("Toaster", "Appliances", 45.00),
        ("Keyboard", "Technology", 75.00), ("Monitor", "Technology", 300.00)
    ]

    for name, category, price in product_data:
        cursor.execute("INSERT INTO products (name, category, price) VALUES (?, ?, ?)", (name, category, price))

    # --- Generate 200 Rows of Sales Data ---
    num_sales = 200
    for _ in range(num_sales):
        # Generate random data for a sale
        customer_id = random.randint(1, len(customer_names))

        # Get a random product to retrieve its price
        product_index = random.randint(0, len(product_data) - 1)
        product_id = product_index + 1  # SQLite IDs are 1-based
        product_price = product_data[product_index][2]

        quantity = random.randint(1, 10)

        # Calculate revenue
        revenue = quantity * product_price

        # Generate a random date within the last year
        start_date = datetime.now() - timedelta(days=365)
        random_date = start_date + timedelta(days=random.randint(0, 365))
        sale_date = random_date.strftime('%Y-%m-%d')

        cursor.execute(
            "INSERT INTO sales (customer_id, product_id, quantity, revenue, sale_date) VALUES (?, ?, ?, ?, ?)",
            (customer_id, product_id, quantity, revenue, sale_date)
        )

    # Commit the changes and close the connection
    conn.commit()
    conn.close()

    print("Successfully created 'sales.db' with 200 sales records.")

if __name__ == '__main__':
    create_and_populate_database()



Existing 'sales.db' file deleted.
Successfully created 'sales.db' with 200 sales records.


In [29]:
import sqlite3
import pandas as pd
import os

def analyze_sales_data():
    """
    Connects to the sales.db database, loads the data into pandas DataFrames,
    merges the tables, and calculates the monthly sales trend.
    """
    db_file = 'sales1.db'
    if not os.path.exists(db_file):
        print(f"Error: The database file '{db_file}' was not found.")
        print("Please run the database creation script first to generate the file.")
        return

    try:
        # Connect to the SQLite database
        conn = sqlite3.connect(db_file)
        print("Successfully connected to sales.db")

        # Read tables into pandas DataFrames
        customers_df = pd.read_sql_query("SELECT * FROM customers", conn)
        products_df = pd.read_sql_query("SELECT * FROM products", conn)
        sales_df = pd.read_sql_query("SELECT * FROM sales", conn)

        # Close the database connection
        conn.close()

        # Merge the DataFrames to create a comprehensive dataset
        # First, merge sales with customers on customer_id
        merged_df = pd.merge(sales_df, customers_df, on='customer_id', how='left')
        # Then, merge the result with products on product_id
        merged_df = pd.merge(merged_df, products_df, on='product_id', how='left')

        # --- Your analysis code starts here ---

        # The KeyError happens here if the 'revenue' column does not exist
        if 'revenue' not in merged_df.columns:
            print("\nError: The 'revenue' column was not found in the sales data.")
            print("This usually happens if the sales.db file was not created with the 'revenue' column.")
            print("Please ensure the `sales.db` file is correctly generated using the latest script.")
            return

        # --- Monthly sales trend ---
        # Convert the 'sale_date' column to datetime objects
        merged_df['sale_date'] = pd.to_datetime(merged_df['sale_date'])

        # Create a new 'month' column to group by, using PeriodIndex for proper sorting
        merged_df['month'] = merged_df['sale_date'].dt.to_period('M')

        # Calculate the sum of revenue for each month
        monthly_sales = merged_df.groupby('month')['revenue'].sum().reset_index()

        # Print the monthly sales trend
        print("\nMonthly Sales Trend:\n")
        print(monthly_sales)

        # --- Sales by customer Segment ---
        print("\nSales by Customer Segment:\n")
        sales_by_segment = merged_df.groupby('segment')['revenue'].sum().reset_index()
        print(sales_by_segment)
        
        # --- Top Products by Revenue ---
        print("\nTop Products by Revenue:\n")
        top_products = merged_df.groupby('product_id')['revenue'].sum().reset_index()
        top_products = top_products.sort_values(by='revenue', ascending=False)
        print(top_products.head(5))

        #---revenue by city--
        print("top selling products by city")
        city_revenue = merged_df.groupby('city')['revenue'].sum().reset_index()
        city_revenue = city_revenue.sort_values(by='revenue', ascending=False)
        print(city_revenue.head(5))



        merged_df.to_csv('sales_analysis.csv', index=False)

    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == '__main__':
    analyze_sales_data()


Successfully connected to sales.db

Monthly Sales Trend:

      month   revenue
0   2024-08  20409.95
1   2024-09  32289.96
2   2024-10  36111.00
3   2024-11  27114.90
4   2024-12  46804.28
5   2025-01  12034.90
6   2025-02  13950.29
7   2025-03  26723.90
8   2025-04  15270.93
9   2025-05  11438.50
10  2025-06  29530.44
11  2025-07  16268.84
12  2025-08   1275.00

Sales by Customer Segment:

       segment    revenue
0     Consumer   55039.70
1    Corporate  111348.14
2  Home Office  122835.05

Top Products by Revenue:

   product_id   revenue
0           1  151200.0
3           4   48150.0
9          10   31200.0
2           3   26250.0
6           7   14100.0
top selling products by city
          city    revenue
0      Chicago  116712.99
2  Los Angeles   64288.65
1      Houston   58583.86
3     New York   42607.00
4      Phoenix    7030.39


In [26]:
merged_df['revenue'] = merged_df['quantity'] * merged_df['price']


In [27]:
merged_df.to_csv('sales_analysis.csv', index=False)


In [28]:
import os
os.getcwd()

'C:\\Users\\HP'